<a href="https://colab.research.google.com/github/dzy82f/Titanic/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
url= 'https://raw.githubusercontent.com/dzy82f/Titanic/main/gender_submission.csv'
df1 = pd.read_csv(url)
df1.head()




,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [ ]:
from google.colab import files
uploaded=files.upload()

import io
df2 = pd.read_csv(io.BytesIO(uploaded['train.csv']))

TypeError: ignored